In [351]:
# Initial imports.
import pandas as pd
import numpy as np
import datetime as dt
from path import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [352]:
# Loading data after database join of seasons/quarters file and Apple's stock file
# https://towardsdatascience.com/4-tricks-you-should-know-to-parse-date-columns-with-pandas-read-csv-27355bb2ad0e#:~:text=By%20default%2C%20date%20columns%20are%20parsed%20using%20the,a%20different%20date%20format%2C%20for%20example%2C%20YYYY-DD-MM%20HH%3AMM%3ASS%3A
file_path =Path("./AAPL_Mock_ML_Open.csv")
apple_df = pd.read_csv(file_path, parse_dates=['Date'])
apple_df.head()

,Date,Season,Quarter,Open,High,Low,Close,Adj Close,Volume,Open_24hr_Change,Gain_Loss_Open
0,1981-02-11,Winter,Q1,0.118304,0.118304,0.117746,0.117746,0.092428,13843200,0.118304,Gain
1,1981-02-12,Winter,Q1,0.117188,0.117188,0.116629,0.116629,0.091552,14560000,-0.001116,Loss
2,1981-02-13,Winter,Q1,0.114955,0.114955,0.113839,0.113839,0.089362,11155200,-0.002233,Loss
3,1981-02-17,Winter,Q1,0.116629,0.117188,0.116629,0.116629,0.091552,12275200,0.001674,Gain
4,1981-02-18,Winter,Q1,0.121652,0.122768,0.121652,0.121652,0.095495,19241600,0.005023,Gain


In [353]:
apple_df['Date'] = pd.to_datetime(apple_df['Date'])
apple_df['Date'] = apple_df['Date'].map(dt.datetime.toordinal)
apple_df.head()

,Date,Season,Quarter,Open,High,Low,Close,Adj Close,Volume,Open_24hr_Change,Gain_Loss_Open
0,723222,Winter,Q1,0.118304,0.118304,0.117746,0.117746,0.092428,13843200,0.118304,Gain
1,723223,Winter,Q1,0.117188,0.117188,0.116629,0.116629,0.091552,14560000,-0.001116,Loss
2,723224,Winter,Q1,0.114955,0.114955,0.113839,0.113839,0.089362,11155200,-0.002233,Loss
3,723228,Winter,Q1,0.116629,0.117188,0.116629,0.116629,0.091552,12275200,0.001674,Gain
4,723229,Winter,Q1,0.121652,0.122768,0.121652,0.121652,0.095495,19241600,0.005023,Gain


In [354]:
# Convert date into an integer since the system was not taking it as a float so that the ML model can read the date - for now.
# https://www.geeksforgeeks.org/convert-floats-to-integers-in-a-pandas-dataframe/
#apple_df['Date'] = apple_df['Date'].astype(int)

In [355]:
# Must remain in order to activate label_binarize
#If in doubt refer to sklearn
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
apple_gain_loss_df = apple_df.copy()
apple_gain_loss_df['Gain_Loss_Open'] = le.fit_transform(apple_gain_loss_df['Gain_Loss_Open'])
apple_gain_loss_df.head()

,Date,Season,Quarter,Open,High,Low,Close,Adj Close,Volume,Open_24hr_Change,Gain_Loss_Open
0,723222,Winter,Q1,0.118304,0.118304,0.117746,0.117746,0.092428,13843200,0.118304,0
1,723223,Winter,Q1,0.117188,0.117188,0.116629,0.116629,0.091552,14560000,-0.001116,1
2,723224,Winter,Q1,0.114955,0.114955,0.113839,0.113839,0.089362,11155200,-0.002233,1
3,723228,Winter,Q1,0.116629,0.117188,0.116629,0.116629,0.091552,12275200,0.001674,0
4,723229,Winter,Q1,0.121652,0.122768,0.121652,0.121652,0.095495,19241600,0.005023,0


In [356]:
# If in doubt look up scikit learn label_binarize
from sklearn.preprocessing import label_binarize

binarized_gain_loss = label_binarize(y=list(apple_gain_loss_df['Gain_Loss_Open']), classes=[1,0])
apple_gain_loss_df = apple_df.copy()
apple_gain_loss_df['Gain_Loss_Open'] = binarized_gain_loss
apple_gain_loss_df.head()

,Date,Season,Quarter,Open,High,Low,Close,Adj Close,Volume,Open_24hr_Change,Gain_Loss_Open
0,723222,Winter,Q1,0.118304,0.118304,0.117746,0.117746,0.092428,13843200,0.118304,1
1,723223,Winter,Q1,0.117188,0.117188,0.116629,0.116629,0.091552,14560000,-0.001116,0
2,723224,Winter,Q1,0.114955,0.114955,0.113839,0.113839,0.089362,11155200,-0.002233,0
3,723228,Winter,Q1,0.116629,0.117188,0.116629,0.116629,0.091552,12275200,0.001674,1
4,723229,Winter,Q1,0.121652,0.122768,0.121652,0.121652,0.095495,19241600,0.005023,1


In [357]:
# Perform binary encoding of Season and Quarter columns
apple_binary_encoded = pd.get_dummies(apple_gain_loss_df, columns=["Season", "Quarter"])
apple_binary_encoded.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Open_24hr_Change,Gain_Loss_Open,Season_Spring,Season_Winter,Quarter_Q1,Quarter_Q2
0,723222,0.118304,0.118304,0.117746,0.117746,0.092428,13843200,0.118304,1,0,1,1,0
1,723223,0.117188,0.117188,0.116629,0.116629,0.091552,14560000,-0.001116,0,0,1,1,0
2,723224,0.114955,0.114955,0.113839,0.113839,0.089362,11155200,-0.002233,0,0,1,1,0
3,723228,0.116629,0.117188,0.116629,0.116629,0.091552,12275200,0.001674,1,0,1,1,0
4,723229,0.121652,0.122768,0.121652,0.121652,0.095495,19241600,0.005023,1,0,1,1,0


In [358]:
# Define features set
X = apple_binary_encoded.copy()
X = X.drop(["Gain_Loss_Open","Open_24hr_Change"], axis=1)
X.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Season_Spring,Season_Winter,Quarter_Q1,Quarter_Q2
0,723222,0.118304,0.118304,0.117746,0.117746,0.092428,13843200,0,1,1,0
1,723223,0.117188,0.117188,0.116629,0.116629,0.091552,14560000,0,1,1,0
2,723224,0.114955,0.114955,0.113839,0.113839,0.089362,11155200,0,1,1,0
3,723228,0.116629,0.117188,0.116629,0.116629,0.091552,12275200,0,1,1,0
4,723229,0.121652,0.122768,0.121652,0.121652,0.095495,19241600,0,1,1,0


In [359]:
# Define the target set.
y = apple_df["Gain_Loss_Open"].ravel()
y[:5]

array(['Gain', 'Loss', 'Loss', 'Gain', 'Gain'], dtype=object)

In [360]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [361]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [378]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=100, random_state=78) 

In [379]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [380]:
# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)

In [381]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,2,3
Actual 1,4,2


In [382]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)

In [383]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,2,3
Actual 1,4,2


Accuracy Score : 0.36363636363636365
Classification Report
              precision    recall  f1-score   support

        Gain       0.33      0.40      0.36         5
        Loss       0.40      0.33      0.36         6

    accuracy                           0.36        11
   macro avg       0.37      0.37      0.36        11
weighted avg       0.37      0.36      0.36        11



In [384]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances

array([0.10076159, 0.12538943, 0.15136465, 0.15604846, 0.12766228,
       0.14755515, 0.13845794, 0.01971412, 0.01255131, 0.01099148,
       0.0095036 ])

In [385]:
# We can sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.15604846173491554, 'Low'),
 (0.15136465416204145, 'High'),
 (0.14755514925413318, 'Adj Close'),
 (0.13845793556334493, 'Volume'),
 (0.12766227887205667, 'Close'),
 (0.12538942973752087, 'Open'),
 (0.10076158612219757, 'Date'),
 (0.0197141161051281, 'Season_Spring'),
 (0.012551306409282507, 'Season_Winter'),
 (0.010991484446496056, 'Quarter_Q1'),
 (0.009503597592883306, 'Quarter_Q2')]